<h1> Battle of Neighborhoods- Capstone Project<br><h3> Data Analysis using Machine Learning algorithms and Foursqaure API to choose best place or city to establish food chain business in North India.<br><h4> Created By: Prateek Vashishtha  <br>Start Date: 07 June 2019

<h3> Importing Required Modules

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install seaborn
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [104]:
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi are 28.6141793, 77.2022662.


In [3]:
# create map of New York using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)
map_delhi

In [4]:
import json

In [5]:
CLIENT_ID = 'KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ' # your Foursquare ID
CLIENT_SECRET = 'ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ
CLIENT_SECRET:ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE


In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 50000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ&client_secret=ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE&v=20180605&ll=28.6141793,77.2022662&radius=50000&limit=100'

In [7]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cff66889fb6b7757ef86932'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bc8f2c7762beee1a8bb3d38-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1fa931735',
         'name': 'Hotel',
         'pluralName': 'Hotels',
         'primary': True,
         'shortName': 'Hotel'}],
       'id': '4bc8f2c7762beee1a8bb3d38',
       'location': {'address': 'Janpath',
        'cc': 'IN',
        'city': 'New Delhi',
        'country': 'India',
        'distance': 2041,
        'formattedAddress': ['Janpath', 'New Delhi 110001', 'Delhi', 'India'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.625548038159426,
          'lng': 77.21866351780929}],
 

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
from pandas.io.json import json_normalize

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Imperial,Hotel,28.625548,77.218664
1,Tamra,Restaurant,28.620543,77.218174
2,Naturals Ice Cream,Ice Cream Shop,28.634455,77.222139
3,Lodhi Gardens (लोधी बाग़) (Lodhi Gardens),Park,28.591424,77.220899
4,The Leela Palace,Hotel,28.579923,77.189291


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
delhi_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

The Imperial
Tamra
Naturals Ice Cream
Lodhi Gardens (लोधी बाग़) (Lodhi Gardens)
The Leela Palace
Indian Accent
The Big Chill Cafe
Varq | वर्क
The Lodhi Hotel
Amour Bistro
Humayun’s Tomb | हुमायूँ का मकबरा (Humayun’s Tomb)
Pandey Paan
Gulati Restaurant
Connaught Place | कनॉट प्लेस (Connaught Place)
ITC Maurya
Jamavar Kashmiri Restaurant @ The Leela
le meridian ,new delhi
Delhi Golf Club
Wenger's
HOTEL SARAVANA BHAVAN
Lantern's
Gurudwara Sri Rakabganj Sahibji
Khan Market | खान मार्केट (Khan Market)
Hauz Khas Village
Cafe Coffee Day
L'Opera
Blue Bar
Taj Palace Hotel
PCO
National Crafts Museum | राष्ट्रीय शिल्प संग्रहालय
The Big Chill Cafe
The Oberoi
Starbucks Coffee
SODABOTTLEOPENERWALA
Library Bar
Kunzum Travel Cafe
Sagar Ratna
Sevilla
Shangri-La's - Eros Hotel
The Claridges
Big Yellow Door
HOTEL SARAVANA BHAVAN
India Gate | इंडिया गेट (India Gate)
Diggin
Olive Bar & Kitchen
Perch
The Big Chill Cafe
PVR Director's Cut
Select Citywalk
National Gallery Of Modern Art | राष्ट्रीय आधुनिक कला 

In [19]:
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Imperial,28.625548,77.218664,The Imperial,28.625548,77.218664,Hotel
1,The Imperial,28.625548,77.218664,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
2,The Imperial,28.625548,77.218664,"The Square, Cafe Coffee Day",28.626640,77.219288,Café
3,The Imperial,28.625548,77.218664,1911,28.625531,77.218715,Mediterranean Restaurant
4,The Imperial,28.625548,77.218664,The Spice Route,28.625577,77.218065,Asian Restaurant


In [20]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan Jyoti),7,7,7,7,7,7
Ambience Mall,47,47,47,47,47,47
Amour Bistro,11,11,11,11,11,11
Bengali Market | बंगाली मार्केट | বাংলা বাজার,15,15,15,15,15,15
Big Chill,47,47,47,47,47,47
Big Yellow Door,14,14,14,14,14,14
Blue Bar,32,32,32,32,32,32
Blue Tokai Coffee Roasters,10,10,10,10,10,10
Cafe Coffee Day,32,32,32,32,32,32


In [21]:
delhi_venues[delhi_venues['Venue Category']=='South Indian Restaurant']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,The Imperial,28.625548,77.218664,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
253,Connaught Place | कनॉट प्लेस (Connaught Place),28.632731,77.220018,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
379,Wenger's,28.633412,77.218292,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
450,HOTEL SARAVANA BHAVAN,28.627041,77.219514,HOTEL SARAVANA BHAVAN,28.627041,77.219514,South Indian Restaurant
540,Hauz Khas Village,28.554311,77.195140,Naivedyam,28.554987,77.195104,South Indian Restaurant
754,Starbucks Coffee,28.632011,77.217731,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
897,Kunzum Travel Cafe,28.553684,77.194368,Naivedyam,28.554987,77.195104,South Indian Restaurant
1047,HOTEL SARAVANA BHAVAN,28.632319,77.216445,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant
1590,Imperfecto,28.554657,77.195092,Naivedyam,28.554987,77.195104,South Indian Restaurant
1673,Khan Chacha | खान चाचा | خان چاچا,28.634202,77.220780,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant


In [22]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [23]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)
map_mumbai

In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 50000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KEW0QGAUCDX2EUWJSVB4JO45JB20GGDW3ZID1YXFDR2V3EMQ&client_secret=ETNUWSZXOMNIICV3IO2AKU5JTXOXBLMTPOEKKDB4GI1TFHFE&v=20180605&ll=18.9387711,72.8353355&radius=50000&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cff69669fb6b775bdcc32f7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b0587caf964a520f1a122e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1fa931735',
         'name': 'Hotel',
         'pluralName': 'Hotels',
         'primary': True,
         'shortName': 'Hotel'}],
       'id': '4b0587caf964a520f1a122e3',
       'location': {'address': 'Apollo Bunder',
        'cc': 'IN',
        'city': 'Mumbai',
        'country': 'India',
        'crossStreet': 'P. Jetha Singh Ramchandani Marg',
        'distance': 1842,
        'formattedAddress': ['Apollo Bunder (P. Jetha Singh Ramchandani Marg)',
         'Mumbai 400001',
         'Mahārāshtra',
         'In

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Taj Mahal Palace & Tower,Hotel,18.922306,72.833578
1,Food for Thought,Café,18.932031,72.831667
2,Starbucks,Coffee Shop,18.932190,72.833959
3,Marine Drive,Scenic Lookout,18.941221,72.823261
4,Nariman Point,Scenic Lookout,18.929183,72.822232


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [37]:
mumbai_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

Taj Mahal Palace & Tower
Food for Thought
Starbucks
Marine Drive
Nariman Point
Shree Thaker Bhojnalay
Trishna
National Centre for the Performing Arts (NCPA)
Natural's Ice Cream Parlour
Jehangir Art Gallery
K Rustoms. Ice Cream
Wankhede Stadium
Trident
New Martin
The Sassy Spoon
Café Mondegar
Woodside Inn
Narayan Dosa
Sea Lounge, Taj Mahal Palace Hotel
Queen's Necklace
Sabyasachi
Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (Prince of Wales Museum of Western India)
Theobroma
PizzaExpress Colaba
Bhagat Tarachand Restaurant
Britannia & Co.
Thai Pavilion
The Oberoi
Bachelorr's Ice Creams
Kala Ghoda Café
Café Basilico
The St. Regis Mumbai
Swati Snacks
Yazdani Bakery
The Malabar Hill Club
Haji Ali Sea Face
Le15 Cafe
Pizza By The Bay
Wasabi by Morimoto
High Street Phoenix
Willingdon Sports Club
The Nutcracker
145 Kala Ghoda
Gateway of India
Priyadarshini Park
Mahalaxmi Race Course (Royal Western India Turf Club)
The Irish House
Café Zoe
Aer Four Seasons Hotel
Worli Sea Face
The Dome
Smoke H

In [49]:
mumbai_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Taj Mahal Palace & Tower,18.922306,72.833578,Taj Mahal Palace & Tower,18.922306,72.833578,Hotel
1,Taj Mahal Palace & Tower,18.922306,72.833578,"Sea Lounge, Taj Mahal Palace Hotel",18.921503,72.833298,Lounge
2,Taj Mahal Palace & Tower,18.922306,72.833578,PizzaExpress Colaba,18.923899,72.833452,Pizza Place
3,Taj Mahal Palace & Tower,18.922306,72.833578,Le15 Cafe,18.923709,72.832735,Café
4,Taj Mahal Palace & Tower,18.922306,72.833578,Wasabi by Morimoto,18.922012,72.833663,Japanese Restaurant


In [39]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
145 Kala Ghoda,42,42,42,42,42,42
Aer Four Seasons Hotel,36,36,36,36,36,36
Ashok Wada Pao,13,13,13,13,13,13
Bachelorr's Ice Creams,44,44,44,44,44,44
Bandstand Promenade,20,20,20,20,20,20
Bhagat Tarachand Restaurant,16,16,16,16,16,16
Bombay Canteen,45,45,45,45,45,45
Bombay Coffee House,33,33,33,33,33,33
Bombay Salad Co.,49,49,49,49,49,49


In [51]:
mumbai_venues[mumbai_venues['Venue Category']=='Lounge']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Taj Mahal Palace & Tower,18.922306,72.833578,"Sea Lounge, Taj Mahal Palace Hotel",18.921503,72.833298,Lounge
88,Food for Thought,18.932031,72.831667,Bottle bar,18.930246,72.833423,Lounge
122,Food for Thought,18.932031,72.831667,Panache,18.934942,72.833739,Lounge
165,Starbucks,18.932190,72.833959,Bottle bar,18.930246,72.833423,Lounge
190,Starbucks,18.932190,72.833959,Panache,18.934942,72.833739,Lounge
230,Nariman Point,18.929183,72.822232,The Eau Bar,18.927070,72.820167,Lounge
248,Nariman Point,18.929183,72.822232,The Trident Club Loung,18.927803,72.820529,Lounge
324,National Centre for the Performing Arts (NCPA),18.925631,72.819861,The Eau Bar,18.927070,72.820167,Lounge
342,Natural's Ice Cream Parlour,18.934892,72.824222,The Dome,18.935014,72.824243,Lounge
424,K Rustoms. Ice Cream,18.933478,72.824995,The Dome,18.935014,72.824243,Lounge


In [89]:
mylist=mumbai_venues.groupby(['Venue Category'])['Venue'].value_counts()

In [92]:
pd.set_option('display.max_rows',len(mylist))

In [93]:
mylist

Venue Category                   Venue                                                                                   
American Restaurant              Between Breads                                                                               5
                                 Villa Vandre                                                                                 3
Antique Shop                     Chor Bazaar (Thieves' Market)                                                                2
Arcade                           Clue Hunt                                                                                    5
                                 A Chocolate Affair                                                                           4
                                 American Express Bakery                                                                      3
                                 Barista Shivaji Park                                                         

In [94]:
mumbai_ir=mumbai_venues[mumbai_venues['Venue Category']=='Indian Restaurant']

In [99]:
mumbai_ir.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
22,Taj Mahal Palace & Tower,18.922306,72.833578,Shamiana,18.921919,72.832748,Indian Restaurant
27,Taj Mahal Palace & Tower,18.922306,72.833578,Sahakari Bhandar Restaurant,18.924767,72.831951,Indian Restaurant
48,Food for Thought,18.932031,72.831667,Fountain Sizzlers Restaurant,18.931366,72.831462,Indian Restaurant
51,Food for Thought,18.932031,72.831667,Mahesh Lunch Home,18.934121,72.833821,Indian Restaurant
52,Food for Thought,18.932031,72.831667,Taste Of Kerala,18.934205,72.833215,Indian Restaurant


In [96]:
mumbai_ir.shape

(386, 7)

In [101]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mumbai_ir['Venue Latitude'], mumbai_ir['Venue Longitude'], mumbai_ir['Venue'], mumbai_ir['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)
    
map_mumbai

In [102]:
delhi_ir=delhi_venues[delhi_venues['Venue Category']=='Indian Restaurant']

In [118]:
# create map of Mumbai using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)


In [120]:
delhi_ir=delhi_venues[delhi_venues['Venue Category'].isin(['Hotel','Restarant','Indian Restaurant','South Indian Restaurant','Chinese Restaurant','Italian Restaurant'])]

In [121]:
# add markers to map
for lat, lng, borough, neighborhood in zip(delhi_ir['Venue Latitude'], delhi_ir['Venue Longitude'], delhi_ir['Venue'], delhi_ir['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)
    
map_delhi